## Read data

In [1]:
# Основные библиотеки
import os
import re
import io
import json
import ast
import time
import tempfile
import subprocess

# Работа с данными
import pandas as pd
from collections import Counter

# Многопоточность
from multiprocessing import Process, Queue

# Прогресс-бар
from tqdm import tqdm

# Контекстный менеджер для перенаправления вывода
from contextlib import redirect_stdout

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

path_to_data = "../data/for_teams"

In [2]:
df_solutions_train = pd.read_excel(f'{path_to_data}/train/solutions.xlsx')
df_tests_train = pd.read_excel(f'{path_to_data}/train/tests.xlsx')
df_tasks_train = pd.read_excel(f'{path_to_data}/train/tasks.xlsx')

In [3]:
df_solutions_train.head(1)

,id,task_id,student_solution,author_comment,author_comment_embedding
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...


# Linter

Проверим компилируется код студента или нет. Если не проходит, то добавляем в df_solutions новые колонки.
```
├── df_solutions_train
│   ├── __id
│   ├── task_id
│   ├── student_solution
│   ├── author_comment
│   ├── author_comment_embedding
│   ├── code_problem // есть проблема с кодом или нет True/False
│   ├── problem_message // какая ошибка
│   ├── line_with_code // какой конкретно код с ошибкой
│   ├── error_type // тип ошибки
```

In [4]:
df_solutions_train.iloc[1]['student_solution']

"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']\ncite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']\n\ncolor = input()\n\nif not(color in logo_project) and not(color in cite_project):\n    print(True)\nelse:\n    print(False)"

In [5]:
import ast
import pandas as pd

# Создаем новые столбцы, если они еще не существуют
if 'code_problem' not in df_solutions_train.columns:
    df_solutions_train['code_problem'] = False
if 'problem_message' not in df_solutions_train.columns:
    df_solutions_train['problem_message'] = ''
if 'line_with_code' not in df_solutions_train.columns:
    df_solutions_train['line_with_code'] = ''
if 'error_type' not in df_solutions_train.columns:
    df_solutions_train['error_type'] = ''

# Проходим по всему DataFrame и проверяем код в student_solution
for index, row in df_solutions_train.iterrows():
    try:
        ast.parse(row['student_solution'])
    except SyntaxError as e:
        df_solutions_train.at[index, 'code_problem'] = True
        df_solutions_train.at[index, 'problem_message'] = str(e)
        df_solutions_train.at[index, 'error_type'] = type(e).__name__
        
        # Получаем строку кода, вызвавшую ошибку
        lines = row['student_solution'].split('\n')
        if e.lineno is not None and 1 <= e.lineno <= len(lines):
            df_solutions_train.at[index, 'line_with_code'] = lines[e.lineno - 1]
        else:
            df_solutions_train.at[index, 'line_with_code'] = ''

df_solutions_train.head(10)

,id,task_id,student_solution,author_comment,author_comment_embedding,code_problem,problem_message,line_with_code,error_type
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...,False,,,
1,14,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.25970256328582764 -1.4550446271896362 0.333...,False,,,
2,15,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.28293243050575256 -1.4774413108825684 0.243...,False,,,
3,16,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.44602400064468384 -0.845210611820221 -0.109...,False,,,
4,17,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Вы забыли поставить двоеточие после условия.,-0.2322002649307251 -1.3137400150299072 -0.383...,True,"expected ':' (<unknown>, line 8)",else,SyntaxError
5,18,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Необходимо использовать одинаковые названия пе...,0.4296543300151825 -0.710841715335846 0.390317...,False,,,
6,19,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Вы некорректно поставили отступы перед функцие...,-0.6812416911125183 -1.4100079536437988 -0.242...,True,expected an indented block after 'else' statem...,print(False),IndentationError
7,20,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Вы забыли поставить закрывающую скобку функции...,-0.19086353480815887 -1.0883759260177612 -0.21...,True,"'(' was never closed (<unknown>, line 9)",print(False,SyntaxError
8,21,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...","Проверьте написание метода, который хотите при...",0.15069107711315155 -0.5297605395317078 -0.038...,True,"invalid syntax (<unknown>, line 6)",if color in logo_project and not in cite_project:,SyntaxError
9,22,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",В данном случае не нужно применять функцию int...,-0.32009613513946533 -1.0578300952911377 0.012...,False,,,


## Tests

Пропускаем код студента через все тесты и добавляем указатель где код упал - в закрытых или открытых данных + тесты.
```
├── df_solutions_train
│   ├── __id
│   ├── task_id
│   ├── student_solution
│   ├── author_comment
│   ├── author_comment_embedding
│   ├── code_problem 
│   ├── problem_message 
│   ├── line_with_code 
│   ├── error_type
│   ├── error_open_test // упал в открытых True/False
│   ├── error_hidden_test //  упал в скрытых True/False
│   ├── test_info //  массив массивов, где во внутреннем массиве лежит информация - что на входе, что ожидалось, что пришло.
```
Код немного поменялся по сравнению с оригинальным ноутбуком - проверяем только код, который компилируется(code_problem = False)

In [6]:
def replace_input_with_values(code, input_values):
    # Считаем количество вызовов input() в коде
    input_calls = len(re.findall(r'input\s*\(\s*\)', code))
    
    # Если нет вызовов input(), возвращаем исходный код
    if input_calls == 0:
        return code
    
    # Если input_values — строка, разделяем её на отдельные значения
    if isinstance(input_values, str):
        input_values_list = input_values.split(';')
    else:
        input_values_list = input_values
    
    # Заменяем каждый вызов input() на соответствующее значение
    if len(input_values_list) > 1:
        for value in input_values_list:
            code = re.sub(r'input\s*\(\s*\)', f'{value}', code, count=1)
    else:
        code = re.sub(r'input\s*\(\s*\)', f'"{input_values}"', code)
    return code

def execute_code_with_timeout(code, input_values, timeout=0.5):
    def target(queue, code, input_values):
        # Заменяем вызовы input() на передачу данных напрямую
        code = replace_input_with_values(code, input_values)     
        # Перенаправляем стандартный вывод в буфер
        buffer = io.StringIO()
        with redirect_stdout(buffer):
            try:
                exec(code)
            except Exception as e:
                queue.put(f"Error: {str(e)}")
                return

        # Получаем результат вывода
        output = buffer.getvalue().strip()
        queue.put(output)

    q = Queue()
    p = Process(target=target, args=(q, code, input_values))
    p.start()
    p.join(timeout)

    if p.is_alive():
        p.terminate()
        p.join()
        return "Timeout"
    else:
        return q.get()

def checkout_tests(solutions, tests):
    # Добавляем колонки error_open и error_closed со значением False по умолчанию
    solutions['error_open_tests'] = False
    solutions['error_closed_tests'] = False
    solutions['test_info'] = [[] for _ in range(len(solutions))]  # Новая колонка для хранения информации о тестах

    # Проходим по каждой строке в solutions с использованием tqdm для отображения прогресса
    for index, row in tqdm(solutions.iterrows(), total=solutions.shape[0], desc="Processing solutions"):
        if not row['code_problem']:
            task_id = row['task_id']
            student_solution = row['student_solution']       

            # Фильтруем tests по task_id
            relevant_tests = tests[tests['task_id'] == task_id]

            # Проходим по каждому тесту с использованием tqdm для отображения прогресса
            for test_index, test_row in relevant_tests.iterrows():
                
                test_id = test_row['id']
                test_input = test_row['input']
                expected_output = test_row['output']
                test_type = test_row['type']

                try:
                    actual_output = execute_code_with_timeout(student_solution, test_input)
                    # Сравниваем результат с ожидаемым выводом 
                    if "Timeout" not in actual_output and "Error" not in actual_output:                    
                        if actual_output != expected_output:
                            if test_type == 'open':
                                solutions.at[index, 'error_open_tests'] = True
                            elif test_type == 'closed':
                                solutions.at[index, 'error_closed_tests'] = True

                            # Добавляем информацию о тесте в колонку test_info
                            test_info = [test_input, expected_output, actual_output]
                            solutions.at[index, 'test_info'].append(test_info)
                except Exception as e:
                    pass

    return solutions

In [7]:
df_solutions_train = checkout_tests(df_solutions_train, df_tests_train)
df_solutions_train.head(3)

Processing solutions: 100%|██████████| 347/347 [00:38<00:00,  9.01it/s]


,id,task_id,student_solution,author_comment,author_comment_embedding,code_problem,problem_message,line_with_code,error_type,error_open_tests,error_closed_tests,test_info
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...,False,,,,True,True,"[[#a7f0ca, True, False], [#e4e3b3, False, True..."
1,14,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.25970256328582764 -1.4550446271896362 0.333...,False,,,,True,True,"[[#a7f0ca, True, False], [#a7f0ca, True, False]]"
2,15,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.28293243050575256 -1.4774413108825684 0.243...,False,,,,True,True,"[[#a7f0ca, True, False], [#a7f0ca, True, False]]"


In [8]:
df_solutions_train.iloc[0]["test_info"]
#input/expected output/actual output

[['#a7f0ca', 'True', 'False'],
 ['#e4e3b3', 'False', 'True'],
 ['#a7a8f0', 'False', 'True'],
 ['#c0ced7', 'False', 'True'],
 ['#a7f0ca', 'True', 'False']]

## Work with text

Добавим описание и авторское решение в датафрейм.

In [9]:
for task_id in df_tasks_train['id']:
    df_solutions_train.loc[df_solutions_train['task_id'] == task_id, 'description'] = df_tasks_train.loc[df_tasks_train['id'] == task_id, 'description'].values[0]
    df_solutions_train.loc[df_solutions_train['task_id'] == task_id, 'author_solution'] = df_tasks_train.loc[df_tasks_train['id'] == task_id, 'author_solution'].values[0]

кажется, что в авторском комментарии нам сейчас не нужна информация об ошибке(мы всегда ее можем вывести из колонок - error_open_tests и error_closed_tests )

In [10]:
# Удаление строк "Ошибка в открытых тестах.", "Ошибка в открытых и скрытых тестах.", "Ошибка в скрытых тестах."
df_solutions_train['author_comment'] = df_solutions_train['author_comment'].str.replace("Ошибка в открытых тестах.", "", regex=False)
df_solutions_train['author_comment'] = df_solutions_train['author_comment'].str.replace("Ошибка в открытых и скрытых тестах.", "", regex=False)
df_solutions_train['author_comment'] = df_solutions_train['author_comment'].str.replace("Ошибка в скрытых тестах.", "", regex=False)

# Удаление лишних пробелов и переносов строк
df_solutions_train['author_comment'] = df_solutions_train['author_comment'].str.strip()
df_solutions_train['author_comment'] = df_solutions_train['author_comment'].str.replace(r'\n+', '\n', regex=True)

In [11]:
df_solutions_train.head(2)

,id,task_id,student_solution,author_comment,author_comment_embedding,code_problem,problem_message,line_with_code,error_type,error_open_tests,error_closed_tests,test_info,description,author_solution
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Обратите внимание на неверный оператор сравнен...,-0.2475823611021042 -1.3317935466766357 0.2535...,False,,,,True,True,"[[#a7f0ca, True, False], [#e4e3b3, False, True...","Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."
1,14,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Обратите внимание на неверный оператор сравнен...,-0.25970256328582764 -1.4550446271896362 0.333...,False,,,,True,True,"[[#a7f0ca, True, False], [#a7f0ca, True, False]]","Реализуйте программу, которая проверит, что цв...","logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4..."


### Самые популярные слова

Используем в промте

In [12]:
# Функция для очистки текста и создания мешка слов
def clean_text(text):
    # Удаляем знаки препинания и приводим к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    words = word_tokenize(text)
    # Удаляем стоп-слова
    stop_words = set(stopwords.words('russian'))
    words = [word for word in words if word not in stop_words]
    return words

# Применяем функцию к колонке author_comment
cleaned_words = df_solutions_train['author_comment'].apply(clean_text)

# Объединяем все слова в один список
all_words = [word for sublist in cleaned_words for word in sublist]

# Считаем частоту слов
word_counts = Counter(all_words)

# Выводим топ-10 самых часто встречающихся слов
print("Топ-10 самых часто встречающихся слов:")
print(word_counts.most_common(10))

# Создаем биграммы (словосочетания из двух слов)
bigrams = ngrams(all_words, 2)
bigram_counts = Counter(bigrams)

# Выводим топ-10 самых часто встречающихся биграмм
print("\nТоп-10 самых часто встречающихся биграмм:")
for bigram, count in bigram_counts.most_common(10):
    print(f"{bigram[0]} {bigram[1]}: {count}")

Топ-10 самых часто встречающихся слов:
[('код', 143), ('условия', 138), ('некорректно', 138), ('ваш', 134), ('выполняет', 115), ('условие', 114), ('ошибка', 110), ('попробуйте', 99), ('изменить', 87), ('задания', 84)]

Топ-10 самых часто встречающихся биграмм:
ваш код: 134
выполняет условия: 93
код некорректно: 92
попробуйте изменить: 86
условия задания: 78
некорректно выполняет: 77
скорректировать ошибку: 61
условие if: 48
задания например: 48
изменить условие: 47


### Создаем словарь, для дообучения yandexGPT

In [13]:
# Функция для создания request
def create_request(row):
    req = (
        "Ты - профессиональный программист, учитель и ментор. Давай очень короткие ответы об ошибках в коде, если они есть.\n"
        f"Я присылаю тебе условие задачи:\n {row['description']}\n" #И его эталонное решение: {row['author_solution']}.\n"
    )
    
    if row['code_problem']:
        req += (
            f"У меня есть мое решение: \n{row['student_solution']}. В нем есть ошибки в коде:\n {row['problem_message']}.\n"
            "Дай короткую подсказку в одном предложении о том, что нужно сделать чтобы решить задачу.\n"
        )
    else:
        req += (
            f"У меня есть мое решение:\n {row['student_solution']}\n Оно не проходит тесты:\n"
        )
        for test in row['test_info']:
            req += (
                f"Я отправляю на вход в свой код {test[0]}, должен получить {test[1]}, но получаю {test[2]}.\n"
            )
        req += (
            "Не присылай исправленный код решения, присылай только короткую подсказку.\n"
            "Используй в речи следующие фразы: ваш код, выполняет условия, код некорректно, попробуйте изменить, условия задания, некорректно выполняет, скорректировать ошибку, условие if, задания например, изменить условие или задавай наводящие вопросы, если нужно обратить внимание на ошибку в коде. \n"
            "Обращайся ко мне лично и на вы.\n"
        )
    
    return req

# Создаем новый датафрейм
new_data = {
    #'id': df_solutions_train['task_id'],    
    'request': df_solutions_train.apply(create_request, axis=1),
    'response': df_solutions_train['author_comment']
}

dataset_for_GPT = pd.DataFrame(new_data)

# Выводим новый датафрейм
print(dataset_for_GPT.shape)
print(dataset_for_GPT.iloc[1].request)

(347, 2)
Ты - профессиональный программист, учитель и ментор. Давай очень короткие ответы об ошибках в коде, если они есть.
Я присылаю тебе условие задачи:
 Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:

Даны два списка logo_project и cite_project с кодами используемых цветов (строки).
В переменную color считывается код цвета (строка). Этот код уже написан.
Программа должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True. 
В остальных случаях программа печатает False. 
У меня есть мое решение:
 logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if not(color in logo_project) and not(color in cite_project):
    print(True)
else:
    print(False)
 Оно не проходит тесты:
Я отправляю на вход в свой код #a7f0ca, должен по

In [14]:
dataset_for_GPT.head(2)

,request,response
0,"Ты - профессиональный программист, учитель и м...",Обратите внимание на неверный оператор сравнен...
1,"Ты - профессиональный программист, учитель и м...",Обратите внимание на неверный оператор сравнен...


по условию request может быть не более 4000 символов, response 2000.

In [15]:
# Подсчитываем количество символов в каждой ячейке
dataset_for_GPT['request_length'] = dataset_for_GPT['request'].apply(len)
dataset_for_GPT['response_length'] = dataset_for_GPT['response'].apply(len)

# Находим максимальное значение для request и response
max_request_length = dataset_for_GPT['request_length'].max()
max_response_length = dataset_for_GPT['response_length'].max()

print(f"Максимальное количество символов в request: {max_request_length}")
print(f"Максимальное количество символов в response: {max_response_length}")

Максимальное количество символов в request: 3946
Максимальное количество символов в response: 415


In [16]:
# Подсчитываем количество символов в каждой ячейке столбца request
dataset_for_GPT['request_length'] = dataset_for_GPT['request'].apply(len)

# Считаем количество записей, где длина request больше 3999 символов
count_request_gt_3999 = dataset_for_GPT[dataset_for_GPT['request_length'] > 3999].shape[0]

print(f"Количество записей в request, где длина больше 3999 символов: {count_request_gt_3999}")

Количество записей в request, где длина больше 3999 символов: 0


In [17]:
# Подсчитываем количество символов в каждой ячейке столбца request
dataset_for_GPT['request_length'] = dataset_for_GPT['request'].apply(len)

# Находим первый request, который больше 3999 символов
first_long_request = dataset_for_GPT[dataset_for_GPT['request_length'] > 3999]['request'].iloc[0] if not dataset_for_GPT[dataset_for_GPT['request_length'] > 3999].empty else None

if first_long_request:
    print(f"Первый request, который больше 3999 символов:\n{first_long_request}")
else:
    print("Нет request, которые больше 3999 символов.")

Нет request, которые больше 3999 символов.


#### Создаем json файл вида
```
[
	{
	"request":df.request,
	"response":df.response
	},
	{
	"request":df.request,
	"response":df.response
	},
]
```

In [18]:
dataset_for_GPT= dataset_for_GPT.drop(columns=['request_length', 'response_length'])
dataset_for_GPT.shape

(347, 2)

In [20]:
# Преобразование DataFrame в список словарей
json_data = dataset_for_GPT.to_dict(orient='records')

# Сохранение в JSON файл
#with open(f'{path_to_data}/train/dataset_for_YAGPT.json', 'w', encoding='utf-8') as f:
    #f.write(pd.DataFrame(json_data).to_json(orient='records', force_ascii=False, indent=4))